In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from src.utils.text_preprocessing import preprocess_text, tokenize
from src.utils.reporting import get_cross_validation_report
import warnings
from tqdm import tqdm
tqdm.pandas()

df = pd.read_csv('data/reviews_excerpt.csv')
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    df['text_pp'] = df['text'].progress_apply(preprocess_text)
    df['text_pp'] = df['text_pp'].progress_apply(lambda row: tokenize(row, stem=True))

C:\Users\Dmitry.Volkov\Anaconda3\envs\support-cases\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
100%|██████████| 12230/12230 [00:15<00:00, 791.46it/s]


In [3]:
import numpy as np
from IPython.display import display
from src.frequency_based_classification import get_frequency_map, get_vector

frequency_map = get_frequency_map(df, 'text_pp', 'score')
scores = df['score'].unique()
df['vector'] = df['text_pp'].progress_apply(lambda tokens: get_vector(tokens, scores, frequency_map))

X, y = np.vstack(df['vector']), df['score'].to_numpy()
weighted_f1, report_df, confusion_df = get_cross_validation_report(X, y, seed=0)
print(weighted_f1)
display(report_df)
display(confusion_df)

100%|██████████| 5/5 [00:09<00:00,  1.82s/it]

0.4074


,precision,recall,f1,support
1.0,0.465763,0.533933,0.497524,2446.0
2.0,0.384476,0.346280,0.364379,2446.0
3.0,0.348294,0.317253,0.332050,2446.0
4.0,0.357993,0.344235,0.350980,2446.0
5.0,0.473704,0.511856,0.492042,2446.0


,Pred 1.0,Pred 2.0,Pred 3.0,Pred 4.0,Pred 5.0
True 1.0,1306,465,288,175,212
True 2.0,620,847,437,282,260
True 3.0,423,415,776,530,302
True 4.0,229,287,471,842,617
True 5.0,226,189,256,523,1252
